In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import plotly.express as px

In [11]:
def create_data_frame(line,surface_type):
    returnVal=None
    ifGroundTruth=False
    if 'Odometry:Pose' in line: 
        line=line.replace('[rosout][INFO]','').strip()
        line=line.split(': Odometry:Pose information ')
        date=line[0].split(' ')[0]
        time=line[0].split(' ')[1]
        x=line[1].split(',')[0].replace('x=','')
        y=line[1].split(',')[1].replace('y=','')
        theta=line[1].split(',')[2].replace('theta=','')
        v=line[1].split(',')[3].replace('velocity=','')
        o=line[1].split(',')[4].replace('omega=','')
        d=line[1].split(',')[5].replace('distance_travelled=','')
        returnVal = {'Date':date,'Time':time, 'X_ODOM':float(x),'Y_ODOM':float(y),'Theta':float(theta),'Velocity':float(v),'Omega':float(o),'Odometry_Distance':float(d),'Surface_Type':surface_type}
    elif ' Scan Data' in line:
        line=line.replace('[rosout][INFO]','').strip()
        line=line.split(': Scan Data: ')
        date=line[0].split(' ')[0]
        time=line[0].split(' ')[1]
        ground_truth=line[1].split('in')[0].replace('Distance Travelled =','')
        ground_truth=ground_truth.replace('Distance Travelled=','')
        gr_co_ordinates=line[1].split('in')[1].split('x=')[1].split('y=')
        gr_time=line[1].split('in')[1].split('seconds')[0]
        returnVal={'Ground_Truth_Distance':float(ground_truth),'X_GR':float(gr_co_ordinates[0]),
                   'Y_GR':float(gr_co_ordinates[1]),'time_taken':abs(float(gr_time))}
        ifGroundTruth=True
    return returnVal,ifGroundTruth


def create_CSV(path,surface_type):
    df_individiual = pd.DataFrame(columns=['Date','Time', 'X_ODOM','Y_ODOM','Theta','Velocity','Omega','Odometry_Distance','X_GR','Y_GR','Ground_Truth_Distance','Error','Surface_Type','time_taken']) 
    with open(path) as f:
            f = f.readlines()
            prevOdoData = None
            groundTruthVal = []
            x_gr = []
            y_gr = []
            time_taken=[]
            errorVal=[]
            for line in f:
                if line.startswith( '[rosout][INFO]' ):
                    if 'inital' in line:
                        continue
                    value_dict,ifGroundTruth = create_data_frame(line,surface_type)
                    if not ifGroundTruth and value_dict != None:
                        if(prevOdoData != None and len(groundTruthVal) > 0):
                            prevOdoData["Ground_Truth_Distance"] = sum(groundTruthVal)/len(groundTruthVal)
                            prevOdoData["X_GR"] = sum(x_gr)/len(x_gr)
                            prevOdoData["Y_GR"] = sum(y_gr)/len(y_gr)
                            prevOdoData["time_taken"] = sum(time_taken)/len(time_taken)
#                             prevOdoData["Error"] = sum(errorVal)/len(errorVal)
                        df_individiual=df_individiual.append(prevOdoData,ignore_index=True)
                        prevOdoData = value_dict
                        groundTruthVal = []
                        errorVal = []
                    elif(value_dict != None):
                        groundTruthVal.append(value_dict['Ground_Truth_Distance'])
                        x_gr.append(value_dict['X_GR'])
                        y_gr.append(value_dict['Y_GR'])
                        time_taken.append(value_dict['time_taken'])
#                         errorVal.append(value_dict['Error'])
    return df_individiual

In [18]:
path="ROOMBA/"
surface=os.listdir( path )
df_all_data = pd.DataFrame(columns=['Date','Time', 'X_ODOM','Y_ODOM','Theta','Velocity','Omega','Odometry_Distance','X_GR','Y_GR','Ground_Truth_Distance','Error','Surface_Type','time_taken']) 
for surface_type in surface:
    files=os.listdir( path+surface_type )
    for file_index in range(len(files)):
        path_test=path+surface_type+'/'+files[file_index]
        df_individiual=create_CSV(path_test,surface_type)
        df_individiual.to_csv('csv_check/'+surface_type+str(file_index)+'.csv')
        df_all_data=df_all_data.append(df_individiual,ignore_index=True)